In [ ]:
!pip install matplotlib
!pip install numpy
# !pip install plotly
!pip install pickle-mixin
# !pip install pytest-shutil

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from XrayLib import xray

with open('xrayobj.pkl', 'rb') as pickle_file:
    a = pickle.load(pickle_file)
    
a.plottiff(show=1)
a.plotreflectivity(showreflect=1)

In [ ]:
DataFit1 = np.genfromtxt('FitResults_fitted.fit', delimiter='\t', skip_header=31)
DataFit2 = np.genfromtxt('FitResults_fitted2.fit', delimiter='\t', skip_header=31)

plt.figure
plt.plot(DataFit1[:,0],DataFit1[:,1],'.')
plt.plot(DataFit1[:,0],DataFit1[:,3],'-')
plt.plot(DataFit2[:,0],DataFit2[:,1],'.')
plt.plot(DataFit2[:,0],DataFit2[:,3],'-')